In [8]:
import numpy as np

In [1]:
import pandas as pd
ab_df = pd.read_csv('C:/yan/ab_data.csv', delimiter=',')
rev_df = pd.read_csv('C:/yan/revenue_data.csv', delimiter=',')
mov_df = pd.read_csv('C:/yan/movie_metadata.csv', delimiter=',')

In [6]:
ctrl_lst = ab_df.user_id[ab_df.group == 'control'].unique()
trtmnt_lst = ab_df.user_id[ab_df.group == 'treatment'].unique()

In [80]:
intersect = np.intersect1d(ctrl_lst, trtmnt_lst)
len(intersect)#размер пересекающейся аудитории

1895

In [18]:
ab_df1 = ab_df[~ab_df.user_id.isin(intersect)]
len(ab_df1.user_id[ab_df1.group == 'control'].unique())#количество уникальных пользователей в группе control

144300

In [26]:
sum(ab_df1.user_id.value_counts() > 1) # количество пользователей совершили более чем 1 сессии за время наблюдений

1999

In [33]:
len(ab_df1.user_id.unique())

288689

In [37]:
ab_df1.converted.sum()/ab_df1.converted.count()#конверсия

0.11963686151475121

In [41]:
ab_df1.converted.count()/ab_df1.converted.sum()#обратная конверсия

8.358627828737385

In [46]:
from scipy import stats
import numpy as np
a =  ab_df1.converted.to_list()
mean, sigma = np.mean(a), np.std(a)

conf_int = stats.norm.interval(0.95, loc=mean, 
    scale=sigma / np.sqrt(len(a)))

In [52]:
np.round(conf_int[0], 3)#нижняя граница доверительного интревала

0.118

In [68]:
rev_df = ab_df1[['user_id', 'group']].drop_duplicates(subset=['user_id']).merge(rev_df, how='left', on='user_id')

In [74]:
rev_control = rev_df.revenue[rev_df.group == 'control'].to_numpy()
rev_treat = rev_df.revenue[rev_df.group == 'treatment'].to_numpy()

In [79]:
from scipy.stats import mannwhitneyu
np.round(mannwhitneyu(rev_treat, rev_control)[1],2)# p-value Манна-Уитни

0.12

In [85]:
mov_df.duration.isna().sum()#пропущенных значений

15

In [87]:
mov_df.duration.mean()

107.2010739856802

In [91]:
np.round(mov_df.duration.fillna(value=mov_df.duration.median()).mean(), 2)#среднее арифметическое по длительности фильма


107.19

In [123]:
mov_df.duration.fillna(value=mov_df.duration.median(), inplace=True)
mov_df['movie_duration_category'] = pd.cut(mov_df.duration, bins=[0, 90 * 0.99999, 120, max(mov_df.duration)], labels=["<90", '90-120', '>120'])

In [126]:
mov_df.columns

Index(['color', 'director_name', 'num_critic_for_reviews', 'duration',
       'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name',
       'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name',
       'movie_title', 'num_voted_users', 'cast_total_facebook_likes',
       'actor_3_name', 'facenumber_in_poster', 'plot_keywords',
       'movie_imdb_link', 'num_user_for_reviews', 'language', 'country',
       'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes',
       'imdb_score', 'aspect_ratio', 'movie_facebook_likes',
       'movie_duration_category'],
      dtype='object')

In [133]:
((mov_df['movie_duration_category'] == '90-120') & (mov_df.title_year == 2008)).sum()#Сколько фильмов с длительностью от 90 минут до 2 часов было выпущено в 2008 году

160

In [150]:
temp = mov_df[['imdb_score', 'plot_keywords']]
temp.dropna(inplace=True)

C:\Users\Dotsenko.Semen\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [156]:
temp.imdb_score[temp.plot_keywords.str.contains('love', case=False)].mean()#средний рейтинг у фильмов категории "love"

6.576

In [173]:

mov_df.budget.apply(lambda x: float(x.replace('$', ''))).median()#медианный бюджет всех представленных фильмов

15000000.0

In [187]:
def find_g(name):
    count = 0
    for a in mov_df.genres:
        if a == name:
            count =+1
    return(count)
    #print(count)

In [188]:
find_g('Comedy|Drama')

1

In [182]:
mov_df.genres

0       Action|Adventure|Fantasy|Sci-Fi
1              Action|Adventure|Fantasy
2             Action|Adventure|Thriller
3                       Action|Thriller
4                           Documentary
                     ...               
5038                       Comedy|Drama
5039       Crime|Drama|Mystery|Thriller
5040              Drama|Horror|Thriller
5041               Comedy|Drama|Romance
5042                        Documentary
Name: genres, Length: 5043, dtype: object

In [177]:
mov_df.genres.apply(find_g('Comedy|Drama'))

NameError: name 'genres' is not defined

In [ ]:
for a in name_genres: